In [2]:
import nltk
import os
import math
from collections import defaultdict
from nltk import FreqDist
import numpy as np
import json
from nltk.probability import FreqDist

STOPWORDS = set(nltk.corpus.stopwords.words('english'))
PORTER_STEMMER = nltk.PorterStemmer()
LANCASTER_STEMMER = nltk.LancasterStemmer()

In [3]:
path = '../Collections'

In [8]:
def preprocessing(doc_path, tokenization, normalization):
    # read all fils 
    with open(doc_path, 'r') as file:
        text = file.read()
        
    # Tokenization
    if tokenization == "Split":
        tokens = text.split()
    else:
        exp_reg = nltk.RegexpTokenizer(r'\d+(?:\.\d+)?x\d+|\d+(?:\.\d+)|\w+(?:-\w+)*|(?:[A-Z]\.)+|\w+')
        tokens = exp_reg.tokenize(text)

    # Remove stopwords
    tokens = [term for term in tokens if term.lower() not in STOPWORDS]

    # Normalization
    if normalization == "Porter":
        tokens = [PORTER_STEMMER.stem(term) for term in tokens]
    elif normalization == "Lancaster":
        tokens = [LANCASTER_STEMMER.stem(term) for term in tokens]
    print("------Tokens:------------------",tokens)
    return tokens

In [11]:
def process_input(query, normalization):
    # appliquer le traitement sur la requete
    if normalization == "Porter":
        query = PORTER_STEMMER.stem(query) 
    elif normalization == "Lancaster":
        query = LANCASTER_STEMMER.stem(query) 
    return query

In [4]:
def create_descriptor_and_inverse_files_with_weights(path, tokenization, normalization, output_path="output"):
    # Créer un nom unique pour les fichiers en fonction des choix de tokenization et normalization
    descriptor_filename = f"descripteur_{tokenization}_{normalization}.json"
    inverse_index_filename = f"inverse_index_{tokenization}_{normalization}.json"
    
    descriptor_path = os.path.join(output_path, descriptor_filename)
    inverse_index_path = os.path.join(output_path, inverse_index_filename)
    
    # Vérifier si les fichiers existent déjà
    if os.path.exists(descriptor_path) and os.path.exists(inverse_index_path):
        print(f"Les fichiers descripteur et inverse existent déjà : {descriptor_path} et {inverse_index_path}")
        
        # Charger les fichiers existants
        with open(descriptor_path, "r", encoding="utf-8") as desc_file:
            descripteur = json.load(desc_file)
        with open(inverse_index_path, "r", encoding="utf-8") as inverse_file:
            inverse_index = json.load(inverse_file)
        
        return descripteur, inverse_index
    
    # Si les fichiers n'existent pas, les créer
    print("Création des fichiers descripteur et inverse...")
    
    # Initialiser les fichiers descripteur et inverse
    descripteur = {}
    inverse_index = defaultdict(lambda: defaultdict(lambda: {"freq": 0, "poids": 0}))
    
    # Nombre total de documents
    documents = os.listdir(path)
    N = len(documents)
    
    # Calcul des fréquences globales pour les poids
    global_term_frequencies = defaultdict(int)
    for doc_name in documents:
        doc_path = os.path.join(path, doc_name)
        tokens = preprocessing(doc_path, tokenization, normalization)
        unique_terms = set(tokens)
        for term in unique_terms:
            global_term_frequencies[term] += 1
    
    # Construire les fichiers descripteur et inverse
    for doc_name in documents:
        doc_path = os.path.join(path, doc_name)
        tokens = preprocessing(doc_path, tokenization, normalization)
        terms_freq = FreqDist(tokens)  # Fréquence des termes
        max_freq = max(terms_freq.values())  # Fréquence maximale dans le document
        doc_key = os.path.splitext(doc_name)[0]
        
        # Ajouter les termes au fichier descripteur
        descripteur[doc_key] = {}
        for term, freq in terms_freq.items():
            poids = (freq / max_freq) * math.log10((N / global_term_frequencies[term]) + 1)
            descripteur[doc_key][term] = {"freq": freq, "poids": round(poids, 4)}
            
            # Ajouter les termes au fichier inverse
            inverse_index[term][doc_key]["freq"] = freq
            inverse_index[term][doc_key]["poids"] = round(poids, 4)
    
    # Sauvegarder les fichiers en JSON
    os.makedirs(output_path, exist_ok=True)
    
    with open(descriptor_path, "w", encoding="utf-8") as desc_file:
        json.dump(descripteur, desc_file, indent=4, ensure_ascii=False)
        
    with open(inverse_index_path, "w", encoding="utf-8") as inverse_file:
        json.dump(inverse_index, inverse_file, indent=4, ensure_ascii=False)
    

    return descripteur, inverse_index



In [5]:
# Exemple d'utilisation
path = "../Collections"  # Remplacez par le chemin de votre dossier contenant les documents
tokenization = "Split"  # Ou "Regex" selon votre choix
normalization = "Porter"  # Ou "Lancaster" selon votre choix
descripteur, inverse_index = create_descriptor_and_inverse_files_with_weights(path, tokenization, normalization)


Les fichiers descripteur et inverse existent déjà : output\descripteur_Split_Porter.json et output\inverse_index_Split_Porter.json


In [9]:
def calculer_relevance_BM25(query, tokenization, normalization, path, k=1.5, b=0.75, output_path="output"):
    # Générer les noms des fichiers
    descriptor_filename = f"descripteur_{tokenization}_{normalization}.json"
    inverse_index_filename = f"inverse_index_{tokenization}_{normalization}.json"

    descriptor_path = os.path.join(output_path, descriptor_filename)
    inverse_index_path = os.path.join(output_path, inverse_index_filename)

    # Vérifier si les fichiers existent, sinon les créer
    if not os.path.exists(descriptor_path) or not os.path.exists(inverse_index_path):
        print(f"Les fichiers pour {tokenization} et {normalization} n'existent pas. Création en cours...")
        create_descriptor_and_inverse_files_with_weights(path, tokenization, normalization, output_path)

    # Charger les fichiers descripteur et inverse
    with open(descriptor_path, "r", encoding="utf-8") as desc_file:
        descripteur = json.load(desc_file)
    with open(inverse_index_path, "r", encoding="utf-8") as inverse_file:
        inverse_index = json.load(inverse_file)
        
    

    # Calcul des longueurs de documents et de la taille moyenne
    doc_lengths = {doc: sum(term_data["freq"] for term_data in terms.values()) for doc, terms in descripteur.items()}

    # Nbre total des terms
    total_terms = sum(doc_lengths.values())
    # Nombre total de documents
    N = len(descripteur) 
    # la taille moyenne des documents 
    avdl = total_terms / N if N > 0 else 1  # Eviter division par zéro
    
    # Initialiser les scores de pertinence
    relevance_dict = {doc: 0 for doc in descripteur}

    # Traiter chaque terme de la requête
    for term in query.split():
        term = process_input(term, normalization)
        
        # dans le cas ou le terme n'exist pas dans le fichier inverse
        if term not in inverse_index:
            continue  # Ignorer les termes absents du fichier inverse

        # Nombre de documents contenant le terme
        ni = len(inverse_index[term])
        # print(f"nombre de document contenant {term} : {ni}")

        # Calculer l'IDF avec un seuil pour éviter des valeurs négatives ou extrêmes
        idf = math.log10(((N - ni + 0.5) / (ni + 0.5)) ) if ni + 0.5 != 0 else 0
        # print(idf)
        for doc_name, data in inverse_index[term].items():
            freq_ti_d = data["freq"]  # Fréquence du terme dans le document
            # print(f"frequant terme {term} dans le document {doc_name} est :{freq_ti_d}")
            dl = doc_lengths.get(doc_name, 0)  # Taille du document d
            
            if dl == 0:
                continue  # Éviter division par zéro

            # Calcul du score BM25
            # numerator = freq_ti_d * (k + 1)
            numerator = freq_ti_d 
            denominator = freq_ti_d + (k * ((1 - b) + b * (dl / avdl)))
            RSV = idf * (numerator / denominator)

            
            # Ajouter au score total du document
            relevance_dict[doc_name] += RSV
            
    # print(relevance_dict)
    
    filtered_relevances = {doc: score for doc, score in relevance_dict.items() if score != 0}

    # Trier les documents par pertinence décroissante
    sorted_relevance = dict(sorted(filtered_relevances.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_relevance


In [12]:
tokenization = "Split"  # Ou "Regex" selon votre choix
normalization = "Lancaster"  # Ou "Lancaster" selon votre choix
query = "effect distribution "  # Exemple de requête
path = "../Collections"  # Chemin vers le dossier contenant les documents
output_path = "output"


relevance_scores = calculer_relevance_BM25(query, tokenization, normalization, path, k=1.5, b=0.75, output_path=output_path)

if relevance_scores:
    print("Scores de pertinence :", relevance_scores)

Scores de pertinence : {'D4': -0.08823075947891026, 'D2': -0.09795556690944766, 'D1': -0.2728925083902616, 'D3': -0.27339867077595187, 'D6': -0.3028044304368359}
